<a href="https://colab.research.google.com/github/Tasfia-007/QOSF-Mentorship-Screeing_Tasks/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Qiskit and qiskit-aer if not already installed
!pip install qiskit --upgrade
!pip install qiskit-aer --upgrade

# Import necessary libraries
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error
import numpy as np

def add_noise(prob_1q: float, prob_2q: float, circuit: QuantumCircuit) -> QuantumCircuit:
    """
    Add random Pauli noise to a quantum circuit.

    Parameters:
    - prob_1q: Probability of applying random Pauli operator after a single-qubit gate.
    - prob_2q: Probability of applying random Pauli operator after a two-qubit gate.
    - circuit: The quantum circuit where the noise will be added.

    Returns:
    - A new quantum circuit with noise applied.
    """
    # Create a new quantum circuit that will include the noise, along with classical registers for measurements
    noisy_circuit = QuantumCircuit(*circuit.qregs, *circuit.cregs)

    # Define the set of Pauli operators: I (Identity), X, Y, Z
    pauli_ops = ['I', 'X', 'Y', 'Z']

    # Loop through the original circuit's gates
    for instr, qargs, cargs in circuit.data:
        # Add the original gate to the new circuit
        noisy_circuit.append(instr, qargs, cargs)

        # Check if it is a 1-qubit gate and apply noise based on probability
        if instr.num_qubits == 1 and np.random.rand() < prob_1q:
            random_pauli = np.random.choice(pauli_ops[1:])  # Ignore 'I', choose X, Y, or Z
            if random_pauli == 'X':
                noisy_circuit.x(qargs[0])  # Apply an X gate
            elif random_pauli == 'Y':
                noisy_circuit.y(qargs[0])  # Apply a Y gate
            elif random_pauli == 'Z':
                noisy_circuit.z(qargs[0])  # Apply a Z gate

        # Check if it is a 2-qubit gate and apply noise based on probability
        elif instr.num_qubits == 2 and np.random.rand() < prob_2q:
            for qubit in qargs:
                random_pauli = np.random.choice(pauli_ops[1:])  # Ignore 'I', choose X, Y, or Z
                if random_pauli == 'X':
                    noisy_circuit.x(qubit)  # Apply an X gate
                elif random_pauli == 'Y':
                    noisy_circuit.y(qubit)  # Apply a Y gate
                elif random_pauli == 'Z':
                    noisy_circuit.z(qubit)  # Apply a Z gate

    return noisy_circuit

# Example of how to use this noise function
qc = QuantumCircuit(2, 2)  # Define a quantum circuit with 2 qubits and 2 classical bits
qc.h(0)   # Apply Hadamard gate to qubit 0 (superposition)
qc.cx(0, 1)  # Apply CNOT gate between qubits 0 and 1 (entanglement)

# Adding measurement to both qubits
qc.measure([0, 1], [0, 1])

# Adding noise with probabilities for 1-qubit and 2-qubit gates
prob_1q = 0.1  # 10% chance of adding noise after single-qubit gate
prob_2q = 0.2  # 20% chance of adding noise after two-qubit gate
noisy_qc = add_noise(prob_1q, prob_2q, qc)

# Display the new noisy circuit
print(noisy_qc)

# Optional: Simulating the noisy circuit
simulator = AerSimulator()
compiled_noisy_circuit = transpile(noisy_qc, simulator)
result = simulator.run(compiled_noisy_circuit, shots=1024).result()

# Get and print the measurement results
counts = result.get_counts()
print(counts)


     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 
{'00': 501, '11': 523}


<ipython-input-5-fa6df9b4381c>:30: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, qargs, cargs in circuit.data:


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def transform_to_gate_basis(circuit: QuantumCircuit) -> QuantumCircuit:
    """
    Transforms a general quantum circuit to the specified gate basis {CX, ID, RZ, SX, X}.

    Parameters:
    - circuit: The input quantum circuit to be transformed.

    Returns:
    - A new quantum circuit transformed to the specified gate basis.
    """
    # Define the gate basis: {CX, ID, RZ, SX, X}
    basis_gates = ['cx', 'id', 'rz', 'sx', 'x']

    # Transpile the circuit to the given gate basis
    transformed_circuit = transpile(circuit, basis_gates=basis_gates)

    return transformed_circuit

# Example usage:
# Create a quantum circuit with a mix of different gates
qc = QuantumCircuit(2)
qc.h(0)        # Hadamard gate (will be decomposed)
qc.cx(0, 1)    # CX gate (already in the basis)
qc.rz(1.57, 1) # RZ gate (already in the basis)
qc.x(0)        # X gate (already in the basis)
qc.sx(1)       # SX gate (already in the basis)

# Adding measurement to both qubits
qc.measure_all()

# Transform the circuit to the desired gate basis
transformed_qc = transform_to_gate_basis(qc)

# Display the transformed circuit
print("Original Circuit:")
print(qc)
print("\nTransformed Circuit to Gate Basis {CX, ID, RZ, SX, X}:")
print(transformed_qc)

# Optional: Simulating the transformed circuit using AerSimulator
simulator = AerSimulator()
compiled_transformed_qc = transpile(transformed_qc, simulator)
result = simulator.run(compiled_transformed_qc, shots=1024).result()

# Get and print the measurement results
counts = result.get_counts()
print("\nSimulation Results:")
print(counts)


Original Circuit:
        ┌───┐        ┌───┐           ░ ┌─┐   
   q_0: ┤ H ├──■─────┤ X ├───────────░─┤M├───
        └───┘┌─┴─┐┌──┴───┴───┐┌────┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Rz(1.57) ├┤ √X ├─░──╫─┤M├
             └───┘└──────────┘└────┘ ░  ║ └╥┘
meas: 2/════════════════════════════════╩══╩═
                                        0  1 

Transformed Circuit to Gate Basis {CX, ID, RZ, SX, X}:
global phase: π/4
        ┌─────────┐┌────┐┌─────────┐        ┌───┐           ░ ┌─┐   
   q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■─────┤ X ├───────────░─┤M├───
        └─────────┘└────┘└─────────┘┌─┴─┐┌──┴───┴───┐┌────┐ ░ └╥┘┌─┐
   q_1: ────────────────────────────┤ X ├┤ Rz(1.57) ├┤ √X ├─░──╫─┤M├
                                    └───┘└──────────┘└────┘ ░  ║ └╥┘
meas: 2/═══════════════════════════════════════════════════════╩══╩═
                                                               0  1 

Simulation Results:
{'11': 287, '01': 254, '10': 250, '00': 233}


In [ ]:
from qiskit import QuantumCircuit
import numpy as np

def qft(circuit, n):
    """Applies the Quantum Fourier Transform (QFT) to the first n qubits in the circuit."""
    for i in range(n):
        # Apply Hadamard gate to qubit i
        circuit.h(i)
        # Apply controlled phase shift to qubits i and j
        for j in range(i + 1, n):
            circuit.cp(np.pi / (2 ** (j - i)), i, j)
    # Reverse the order of qubits
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

def inverse_qft(circuit, n):
    """Applies the inverse Quantum Fourier Transform (QFT†) to the first n qubits in the circuit."""
    # Reverse the order of qubits back
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)
    # Apply inverse of the controlled phase shifts and Hadamard gates
    for i in reversed(range(n)):
        for j in reversed(range(i + 1, n)):
            circuit.cp(-np.pi / (2 ** (j - i)), i, j)
        circuit.h(i)

def quantum_sum(a: int, b: int, num_bits: int):
    """
    Implements the Draper adder algorithm to add two integers a and b using quantum registers and the QFT.

    Parameters:
    - a: The first integer to add.
    - b: The second integer to add.
    - num_bits: Number of bits needed to represent the integers.

    Returns:
    - QuantumCircuit: The quantum circuit that performs the sum of a and b.
    """
    # Create a quantum circuit with num_bits qubits for the numbers and an additional num_bits for the result
    qc = QuantumCircuit(2 * num_bits)

    # Initialize the qubits in the state representing the binary value of 'a'
    a_bin = bin(a)[2:].zfill(num_bits)
    b_bin = bin(b)[2:].zfill(num_bits)

    # Apply X gates to initialize the value of 'a'
    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(i)

    # Apply X gates to initialize the value of 'b'
    for i, bit in enumerate(reversed(b_bin)):
        if bit == '1':
            qc.x(num_bits + i)

    # Apply the Quantum Fourier Transform to the first register (the one containing 'a')
    qft(qc, num_bits)

    # Perform controlled phase rotations using the second register (containing 'b')
    for i in range(num_bits):
        for j in range(i, num_bits):
            angle = np.pi / (2 ** (j - i + 1))
            qc.cp(angle, num_bits + j, i)

    # Apply the inverse Quantum Fourier Transform to the first register
    inverse_qft(qc, num_bits)

    # Return the circuit with the Draper adder
    return qc

# Example usage:
# Adding two numbers, 3 and 2, using the Draper Adder

# Numbers to add
a = 3
b = 2

# Number of bits (enough to represent both numbers)
num_bits = 3

# Build the Draper Adder circuit
qc = quantum_sum(a, b, num_bits)

# Draw the circuit
print("Quantum Circuit for Draper Adder to add", a, "and", b)
print(qc)


Quantum Circuit for Draper Adder to add 3 and 2
     ┌───┐┌───┐                                                          »
q_0: ┤ X ├┤ H ├─■────────■───────────────────────────X──■────────■───────»
     ├───┤└───┘ │P(π/2)  │       ┌───┐               │  │        │       »
q_1: ┤ X ├──────■────────┼───────┤ H ├─■─────────────┼──┼────────┼───────»
     └───┘               │P(π/4) └───┘ │P(π/2) ┌───┐ │  │        │       »
q_2: ────────────────────■─────────────■───────┤ H ├─X──┼────────┼───────»
                                               └───┘    │P(π/2)  │       »
q_3: ───────────────────────────────────────────────────■────────┼───────»
     ┌───┐                                                       │P(π/4) »
q_4: ┤ X ├───────────────────────────────────────────────────────■───────»
     └───┘                                                               »
q_5: ────────────────────────────────────────────────────────────────────»
                                                    

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error
import numpy as np

# QFT function
def qft(circuit, n):
    for i in range(n):
        circuit.h(i)
        for j in range(i + 1, n):
            circuit.cp(np.pi / (2 ** (j - i)), i, j)
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

# Inverse QFT function
def inverse_qft(circuit, n):
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)
    for i in reversed(range(n)):
        for j in reversed(range(i + 1, n)):
            circuit.cp(-np.pi / (2 ** (j - i)), i, j)
        circuit.h(i)

# Draper Adder function
def quantum_sum(a: int, b: int, num_bits: int):
    qc = QuantumCircuit(2 * num_bits)
    a_bin = bin(a)[2:].zfill(num_bits)
    b_bin = bin(b)[2:].zfill(num_bits)

    # Initialize the first register with the binary value of 'a'
    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(i)

    # Initialize the second register with the binary value of 'b'
    for i, bit in enumerate(reversed(b_bin)):
        if bit == '1':
            qc.x(num_bits + i)

    # Apply the QFT to the first register
    qft(qc, num_bits)

    # Controlled phase rotations using the second register
    for i in range(num_bits):
        for j in range(i, num_bits):
            angle = np.pi / (2 ** (j - i + 1))
            qc.cp(angle, num_bits + j, i)

    # Apply the inverse QFT to the first register
    inverse_qft(qc, num_bits)

    return qc

# Gate Basis Transformation
def transform_to_gate_basis(circuit: QuantumCircuit) -> QuantumCircuit:
    basis_gates = ['cx', 'id', 'rz', 'sx', 'x']
    transformed_circuit = transpile(circuit, basis_gates=basis_gates)
    return transformed_circuit

# Noise Model
def add_noise(prob_1q: float, prob_2q: float, circuit: QuantumCircuit) -> QuantumCircuit:
    noisy_circuit = QuantumCircuit(*circuit.qregs, *circuit.cregs)
    pauli_ops = ['I', 'X', 'Y', 'Z']

    for instr, qargs, cargs in circuit.data:
        noisy_circuit.append(instr, qargs, cargs)
        if instr.num_qubits == 1 and np.random.rand() < prob_1q:
            random_pauli = np.random.choice(pauli_ops[1:])
            if random_pauli == 'X':
                noisy_circuit.x(qargs[0])
            elif random_pauli == 'Y':
                noisy_circuit.y(qargs[0])
            elif random_pauli == 'Z':
                noisy_circuit.z(qargs[0])
        elif instr.num_qubits == 2 and np.random.rand() < prob_2q:
            for qubit in qargs:
                random_pauli = np.random.choice(pauli_ops[1:])
                if random_pauli == 'X':
                    noisy_circuit.x(qubit)
                elif random_pauli == 'Y':
                    noisy_circuit.y(qubit)
                elif random_pauli == 'Z':
                    noisy_circuit.z(qubit)

    return noisy_circuit

# Simulation and Analysis
def analyze_quantum_addition(a, b, num_bits, prob_1q, prob_2q):
    # Build the quantum circuit
    qc = quantum_sum(a, b, num_bits)

    # Transform the circuit to the gate basis
    transformed_qc = transform_to_gate_basis(qc)

    # Add noise to the circuit
    noisy_qc = add_noise(prob_1q, prob_2q, transformed_qc)

    # Add measurement
    noisy_qc.measure_all()

    # Simulate the noisy circuit
    simulator = AerSimulator()
    compiled_noisy_circuit = transpile(noisy_qc, simulator)
    result = simulator.run(compiled_noisy_circuit, shots=1024).result()

    # Get and print the measurement results
    counts = result.get_counts()
    print(f"\nResults for a = {a}, b = {b} with noise (1-qubit noise={prob_1q}, 2-qubit noise={prob_2q}):")
    print(counts)

# Run the analysis for different noise levels
a = 3
b = 2
num_bits = 3
print("No noise:")
analyze_quantum_addition(a, b, num_bits, 0, 0)  # No noise
print("\nLow noise:")
analyze_quantum_addition(a, b, num_bits, 0.01, 0.05)  # Low noise
print("\nHigh noise:")
analyze_quantum_addition(a, b, num_bits, 0.1, 0.2)  # High noise


No noise:

Results for a = 3, b = 2 with noise (1-qubit noise=0, 2-qubit noise=0):
{'010110': 20, '010000': 221, '010111': 138, '010011': 428, '010001': 217}

Low noise:


<ipython-input-13-72902966c3fe>:66: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, qargs, cargs in circuit.data:
<ipython-input-13-72902966c3fe>:66: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, qargs, cargs in circuit.data:



Results for a = 3, b = 2 with noise (1-qubit noise=0.01, 2-qubit noise=0.05):
{'000111': 64, '000001': 71, '000000': 85, '000100': 392, '000110': 352, '000101': 60}

High noise:

Results for a = 3, b = 2 with noise (1-qubit noise=0.1, 2-qubit noise=0.2):
{'110110': 23, '110111': 122, '110100': 147, '110101': 732}


<ipython-input-13-72902966c3fe>:66: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, qargs, cargs in circuit.data:
